<a href="https://colab.research.google.com/github/elangbijak4/Quantum_Algorithm_Almudaya/blob/main/Quantum_PoS_Blockchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install cirq

In [3]:
import datetime # Import the datetime module
import random
import cirq

In [9]:
import hashlib

In [4]:
class Block:
    def __init__(self, index, timestamp, data, previous_hash, nonce=0, validator=None):
        self.index = index
        self.timestamp = timestamp
        self.data = data
        self.previous_hash = previous_hash
        self.nonce = nonce
        self.validator = validator
        self.hash = self.calculate_hash()

    def calculate_hash(self):
        sha = hashlib.sha256()
        sha.update(str(self.index).encode('utf-8') +
                   str(self.timestamp).encode('utf-8') +
                   str(self.data).encode('utf-8') +
                   str(self.previous_hash).encode('utf-8') +
                   str(self.nonce).encode('utf-8') +
                   str(self.validator).encode('utf-8'))
        return sha.hexdigest()

In [6]:
class QuantumProofOfStake:
    def __init__(self, stakeholders):
        self.stakeholders = stakeholders

    def quantum_random_selection(self):
        # Buat sirkuit kuantum untuk memilih validator berdasarkan stake
        qubits = cirq.LineQubit.range(len(self.stakeholders))

        circuit = cirq.Circuit(
            cirq.H.on_each(*qubits),  # Hadamard gate untuk menciptakan superposisi
            cirq.measure(*qubits, key='result')  # Ukur hasil superposisi
        )

        simulator = cirq.Simulator()
        result = simulator.run(circuit, repetitions=1)

        # Hitung probabilitas masing-masing pemilik stake dipilih
        prob_distribution = {}
        for i, stakeholder in enumerate(self.stakeholders):
            measurement_result = result.measurements['result'][0][i]
            if measurement_result == 1:
                prob_distribution[stakeholder] = True
            else:
                prob_distribution[stakeholder] = False

        return prob_distribution

    def validate_stake(self, block):
        # Validasi stake: Blok harus memiliki indeks yang merupakan kelipatan dari nilai stake
        return block.index % len(self.stakeholders) == 0

In [12]:
class Blockchain:
    def __init__(self, stakeholders):
        self.chain = [self.create_genesis_block()] # Create the genesis block
        self.qpos = QuantumProofOfStake(stakeholders)

    def create_genesis_block(self):
        return Block(0, datetime.datetime.now(), "Genesis Block", "0") # Use datetime.datetime.now()

    def get_latest_block(self):
        return self.chain[-1]

    def add_block(self, new_block):
        new_block.previous_hash = self.get_latest_block().hash

        # Pilih validator menggunakan sirkuit kuantum
        prob_distribution = self.qpos.quantum_random_selection()
        selected_validator = random.choices(list(prob_distribution.keys()), weights=list(prob_distribution.values()))[0]

        # Adjust block index for stake validation if needed
        while not self.qpos.validate_stake(new_block):
            new_block.index += 1  # Increment index until valid

        # Simpan validator yang dipilih sebagai bagian dari blok
        new_block.validator = selected_validator

        # Hitung hash dan tambahkan blok ke blockchain
        new_block.hash = new_block.calculate_hash()
        self.chain.append(new_block)
        print(f"Block mined with Validator: {new_block.validator}")

In [13]:
# Contoh penggunaan dalam sebuah blockchain dengan stakeholders
stakeholders = ['Alice', 'Bob', 'Charlie', 'David']
my_blockchain = Blockchain(stakeholders)

# Menambahkan beberapa blok ke blockchain menggunakan Quantum Proof of Stake (QPoS)
for i in range(1, 6):
    new_block = Block(i, datetime.datetime.now(), {'amount': random.randint(1, 10)}, '') # Use datetime.datetime.now()
    my_blockchain.add_block(new_block)

Block mined with Validator: David
Block mined with Validator: Charlie
Block mined with Validator: Alice
Block mined with Validator: Bob
Block mined with Validator: Charlie
